In [1]:
import os


os.environ['SPARK_NAME'] = "/opt/spark"
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/venv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/venv/bin/python3'

In [2]:
from pyspark.sql import SparkSession


spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","4g")\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/27 12:28:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[2] appName=Create-DataFrame>

**Zadanie 1**  
Pamiętacie plik zamówienia.txt ?
Plik został umieszczony w folderze z labem w repozytorium.

Wczytaj ten plik za pomocą Sparka do dowolnego typu danych (RDD, Spark DataFrame) i dokonaj transformacji tak aby:
* naprawić problemy z kodowaniem znaków (replace?) w kolumnie Sprzedawca
* poprawić format danych w kolumnie Utarg
* dodać odpowiednie typy danych
* kolumna idZamowienia powinna być traktowana jako klucz (indeks)

In [3]:
sc = spark.sparkContext
df = spark.read.csv('./data/zamowienia.txt', header=True, sep=';', encoding='UTF-8')
df.printSchema()

root
 |-- Kraj: string (nullable = true)
 |-- Sprzedawca: string (nullable = true)
 |-- Data zamowienia: string (nullable = true)
 |-- idZamowienia: string (nullable = true)
 |-- Utarg: string (nullable = true)



In [4]:
from pyspark.sql.functions import col, regexp_replace, to_date
from pyspark.sql.types import IntegerType, DoubleType

df = df.withColumn("Sprzedawca", regexp_replace(col("Sprzedawca"), "[^\\w\\s]", ""))

df = df.withColumn("Utarg", regexp_replace(col("Utarg"), "[^0-9,]", ""))
df = df.withColumn("Utarg", regexp_replace(col("Utarg"), ",", "."))

df = df.withColumn("idZamowienia", col("idZamowienia").cast(IntegerType()))
df = df.withColumn("Data zamowienia", to_date(col("Data zamowienia"), "dd.MM.yyyy"))
df = df.withColumn("Utarg", col("Utarg").cast(DoubleType()))

In [5]:
df.createOrReplaceTempView("ORDERS_DATA")
spark.catalog.listTables()

[Table(name='ORDERS_DATA', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [6]:
df.printSchema()

root
 |-- Kraj: string (nullable = true)
 |-- Sprzedawca: string (nullable = true)
 |-- Data zamowienia: date (nullable = true)
 |-- idZamowienia: integer (nullable = true)
 |-- Utarg: double (nullable = true)



In [7]:
spark.sql("Select * from ORDERS_DATA limit 10").show()

+------+----------+---------------+------------+------+
|  Kraj|Sprzedawca|Data zamowienia|idZamowienia| Utarg|
+------+----------+---------------+------------+------+
|Polska|  Kowalski|     2003-07-16|       10248| 440.0|
|Polska|   Sowiski|     2003-07-10|       10249|1863.4|
|Niemcy|   Peacock|     2003-07-12|       10250|1552.6|
|Niemcy| Leverling|     2003-07-15|       10251|654.06|
|Niemcy|   Peacock|     2003-07-11|       10252|3597.9|
|Niemcy| Leverling|     2003-07-16|       10253|1444.8|
|Polska|  Kowalski|     2003-07-23|       10254|556.62|
|Polska|     Dudek|     2003-07-15|       10255|2490.5|
|Niemcy| Leverling|     2003-07-17|       10256| 517.8|
|Niemcy|   Peacock|     2003-07-22|       10257|1119.9|
+------+----------+---------------+------------+------+

